In [0]:
## NOTE: Run this cell only once!! The dependencies are installed in the VM instance
##       The same VM instance is attached to the colab notebook

## Install the following dependencies
##  - Pytorch 0.4.1
##  - torchvision
##  - visdom
##  - localtunnel

!pip3 install http://download.pytorch.org/whl/cu92/torch-0.4.1-cp36-cp36m-linux_x86_64.whl
!pip3 install torchvision
!pip install visdom
!npm install -g localtunnel

In [0]:
## Mount gdrive location
# from google.colab import drive
# drive.mount('/content/gdrive')

## Writing Files to google drive
# with open('./gdrive/My \Drive/COMPSCI_689/test_data.csv', 'w') as fp:
#   fp.write('Hello\n')

# with open('./gdrive/My \Drive/COMPSCI_689/test_data.csv', 'r') as fp:
#   for line in fp.readlines():
#     print('line=', line)

In [0]:
## Setting up visdom server for creating visualizations
import time


port = 6006
get_ipython().system_raw('/usr/bin/python3 -m visdom.server -port ' + str(port) + ' >> visdomlog.txt 2>&1 &')
get_ipython().system_raw('lt --port ' + str(port) + ' >> url.txt 2>&1 &')

time.sleep(10)  # Giving time to the visdom server to get started
! cat visdomlog.txt
! cat ./url.txt

In [0]:
## Cleaning up connections to visdom server

# !rm visdomlog.txt
# !rm url.txt
# !ps -eaf | grep visdom
# !kill -9 <visdom server pid>

In [0]:
# Scatter plot example class for visdom
import numpy as np
from visdom import Visdom

          
class ScatterPlot(object):
  def __init__(self, xlabel, ylabel, port=8080):
    self.viz = Visdom(port=port)
    self.xlabel = xlabel
    self.ylabel = ylabel
    self.window = {}
    
  def scatter_plot(self, name, x, y):
    if name not in self.window.keys():
      win = self.viz.scatter(
              X = np.array([x, y]).reshape(1, -1),
              opts = dict(
                  title = name,
                  xlabel = self.xlabel,
                  ylabel = self.ylabel
                )
            )
      self.window[name] = win
    else:
      self.viz.scatter(
          X = np.array([x, y]).reshape(1, -1),
          update='append',
          win=self.window[name]
      )
  

In [0]:
## Training a CNN model on MNIST using PyTorch

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
      
      
      
## Train and Test functions

# Creating plotting object
plt = ScatterPlot(xlabel='Epochs', ylabel='Loss', port=port)

def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    
    epoch_loss = 0.0
    num_iterations = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        
        loss = F.nll_loss(output, target)
        epoch_loss += loss.data.cpu().numpy()
        
        loss.backward()
        optimizer.step()
        if batch_idx % args['log_interval'] == 0:
          print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
              epoch, batch_idx * len(data), len(train_loader.dataset),
              100. * batch_idx / len(train_loader), loss.item()))
        num_iterations += 1

    plt.scatter_plot(name='Training Loss', x=epoch, y=epoch_loss/num_iterations)

    
def test(args, model, device, test_loader, epoch):
    model.eval()
    
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    plt.scatter_plot(name='Testing Loss', x=epoch, y=test_loss)

In [0]:
## Main function
import torch

def main():
  use_cuda = True
  device = torch.device("cuda" if use_cuda else "cpu")
  
  data_path = '../data'  ## Path where the data is stored in the VM instance
  
  num_epochs = 5
  
  args = {
      'batch_size': 64,
      'test_batch_size': 1000,
      'log_interval': 100
  }
  
  kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
  train_loader = torch.utils.data.DataLoader(
      datasets.MNIST(data_path, train=True, download=True,
                     transform=transforms.Compose([
                         transforms.ToTensor(),
                         transforms.Normalize((0.1307,), (0.3081,))
                     ])),
      batch_size=args['batch_size'], shuffle=True, **kwargs)
  test_loader = torch.utils.data.DataLoader(
      datasets.MNIST(data_path, train=False, transform=transforms.Compose([
                         transforms.ToTensor(),
                         transforms.Normalize((0.1307,), (0.3081,))
                     ])),
      batch_size=args['test_batch_size'], shuffle=True, **kwargs)
  
  model = Net().to(device)
  optimizer = optim.Adam(model.parameters())
  
  for epoch in range(1, num_epochs + 1):
    train(args, model, device, train_loader, optimizer, epoch)
    test(args, model, device, test_loader, epoch)

In [0]:
main()